In [1]:
import multiprocessing
import os

import cv2
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from enum import Enum
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from albumentations import *
from albumentations.pytorch import ToTensorV2

## Config

In [2]:
# config
class Config():
    seed = 42
    
    # 경로
    data_dir = './input/data/eval'
    # 모델 경로
    save_model_dir = './newModel_exp'
    # Output 경로
    output_dir = './newModel_output'
    resize = [224, 224]
    
    # 추론 설정
    batch_size = 1000
#     n_splits = 5 # k - fold

config = Config()

 # -- settings
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

## test dataset

In [3]:
class AgeLabels(int, Enum):
    YOUNG = 0
    MIDDLE = 1
    OLD = 2
    
    @classmethod
    def class2number(cls, value: int) -> int:
        if value <= 2:
            return cls.YOUNG
        elif value <= 8:
            return cls.MIDDLE
        else:
            return cls.OLD
        
    @classmethod
    def from_cls(cls, value: str) -> int:
        try:
            value = int(value)
        except Exception:
            raise ValueError(f"Age value should be numeric, {value}")

        if value < 20:
            return 0
        elif value < 25:
            return 1
        elif value < 30:
            return 2
        elif value < 35:
            return 3
        elif value < 40:
            return 4
        elif value < 45:
            return 5
        elif value < 50:
            return 6
        elif value < 55:
            return 7
        elif value < 60:
            return 8
        else:
            return 

In [4]:
class TestDataset(Dataset):
    def __init__(self, img_paths, resize, mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246)):
        self.img_paths = img_paths
        self.transform = Compose([
            CenterCrop(height=460, width=320),
            Resize(resize[0], resize[1], p=1.0),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)

    def __getitem__(self, index):
        # image = Image.open(self.img_paths[index])
        image_path = self.img_paths[index]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            image = self.transform(image=np.array(image))['image']
        return image

    def __len__(self):
        return len(self.img_paths)

In [5]:
img_root = os.path.join(config.data_dir, 'images')
info_path = os.path.join(config.data_dir, 'info.csv')
info = pd.read_csv(info_path)
img_paths = [os.path.join(img_root, img_id) for img_id in info.ImageID]

dataset = TestDataset(img_paths, config.resize)
loader = DataLoader(
    dataset,
    batch_size=config.batch_size,
    num_workers=multiprocessing.cpu_count() // 2,
    shuffle=False,
    pin_memory=use_cuda,
    drop_last=False,
)

## Model

In [6]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x
    
class SwinMultiheadModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.base_model = models.swin_b(weights='DEFAULT')
        self.base_model.head = Identity()

        self.fc_mask_head = nn.Sequential(
            nn.Linear(in_features=1024, out_features=1000, bias=True),
            nn.BatchNorm1d(1000),
            nn.ELU(True),
            nn.Dropout(0.5, inplace=True),
            nn.Linear(in_features=1000, out_features=3, bias=True)
        )
        self.fc_gender_head = nn.Sequential(
            nn.Linear(in_features=1024, out_features=1000, bias=True),
            nn.BatchNorm1d(1000),
            nn.ELU(True),
            nn.Dropout(0.5, inplace=True),
            nn.Linear(in_features=1000, out_features=2, bias=True)
        )
    def forward(self, x):
        x = self.base_model(x)
        mask = self.fc_mask_head(x)
        gender = self.fc_gender_head(x)
        return mask, gender

class SwinAgeModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.base_model = models.swin_b(weights='DEFAULT')
        self.base_model.head = nn.Sequential(
            nn.Linear(in_features=1024, out_features=1000, bias=True),
            nn.BatchNorm1d(1000),
            nn.ELU(True),
            nn.Dropout(0.5, inplace=True),
            nn.Linear(in_features=1000, out_features=1000, bias=True),
            nn.BatchNorm1d(1000),
            nn.ELU(True),
            nn.Dropout(0.5, inplace=True),
            nn.Linear(in_features=1000, out_features=10, bias=True)
        )
    def forward(self, x):
        x = self.base_model(x)
        return x

class ResMultiheadModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.base_model = models.resnext101_32x8d(weights=models.ResNeXt101_32X8D_Weights.DEFAULT)
        self.base_model.fc = Identity()

        self.fc_mask = nn.Sequential(
            nn.Linear(in_features=2048, out_features=1000, bias=True),
            nn.BatchNorm1d(1000),
            nn.ELU(True),
            nn.Dropout(0.5, inplace=True),
            nn.Linear(in_features=1000, out_features=3, bias=True)
        )
        self.fc_gender = nn.Sequential(
            nn.Linear(in_features=2048, out_features=1000, bias=True),
            nn.BatchNorm1d(1000),
            nn.ELU(True),
            nn.Dropout(0.5, inplace=True),
            nn.Linear(in_features=1000, out_features=2, bias=True)
        )
    def forward(self, x):
        x = self.base_model(x)
        mask = self.fc_mask(x)
        gender = self.fc_gender(x)
        return mask, gender

class ResAgeModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.base_model = models.resnext101_32x8d(weights=models.ResNeXt101_32X8D_Weights.DEFAULT)
        self.base_model.fc = nn.Sequential(
            nn.Linear(in_features=2048, out_features=1000, bias=True),
            nn.BatchNorm1d(1000),
            nn.ELU(True),
            nn.Dropout(0.5, inplace=True),
            nn.Linear(in_features=1000, out_features=1000, bias=True),
            nn.BatchNorm1d(1000),
            nn.ELU(True),
            nn.Dropout(0.5, inplace=True),
            nn.Linear(in_features=1000, out_features=10, bias=True)
        )
    def forward(self, x):
        x = self.base_model(x)
        return x

In [7]:
model_multi = ResMultiheadModel()
model_age = ResAgeModel()
model_multi_path = os.path.join(config.save_model_dir, f'best_total_multi.pth') #best_age.pth
model_multi.load_state_dict(torch.load(model_multi_path, map_location=device))
model_age_path = os.path.join(config.save_model_dir, f'best_total_age.pth') 
model_age.load_state_dict(torch.load(model_age_path, map_location=device))

model_multi = model_multi.to(device)
model_age = model_age.to(device)

## Inference

In [8]:
model_multi.eval()
model_age.eval()
preds = []
with torch.no_grad():    
    for images in loader:
        images = images.to(device)
        mask_outs, gender_outs = model_multi(images)
        age_outs = model_age(images)
        mask_preds = torch.argmax(mask_outs, dim=-1).cpu()
        age_preds = torch.argmax(age_outs, dim=-1).cpu()
        gender_preds = torch.argmax(gender_outs, dim=-1).cpu()
        
        age_preds = age_preds.apply_(AgeLabels.class2number)

        pred = mask_preds * 6 + gender_preds * 3 + age_preds
        preds.extend(pred.cpu().numpy())

info['ans'] = preds
os.makedirs(config.output_dir)
save_path = os.path.join(config.output_dir, f'output.csv')
info.to_csv(save_path, index=False)
print(f"Inference Done! Inference result saved at {save_path}")

Inference Done! Inference result saved at ./newModel_output/output.csv
